In [0]:
from nltk.corpus import movie_reviews
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, KFold
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import os
import pandas as pd
import string
from string import punctuation
from tqdm.auto import tqdm

![photo](https://i.ibb.co/sHq1M9Z/Kaggle-Leaderboard.png)

Я воспользовался Kaggle API, чтобы загрузить небходимые данные с соревнования. Свои ключ и логин я заменил на другие

In [2]:
os.environ['KAGGLE_USERNAME'] = "username"
os.environ['KAGGLE_KEY'] = "key"

!kaggle competitions download -c simplesentiment

  0% 0.00/2.83k [00:00<?, ?B/s]
100% 2.83k/2.83k [00:00<00:00, 2.41MB/s]
  0% 0.00/50.9k [00:00<?, ?B/s]
100% 50.9k/50.9k [00:00<00:00, 54.1MB/s]
  0% 0.00/193k [00:00<?, ?B/s]
100% 193k/193k [00:00<00:00, 56.4MB/s]


Загружаем данные, с которыми будем работать при обучении

In [3]:
train_data = pd.read_csv('products_sentiment_train.tsv', sep='\t', header=None)
train_data.head()

,0,1
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


Разделяем данные на текста и лейблы

In [0]:
X = train_data.values[:, 0]
y = train_data[1].values

Проверим, не является ли выборка несбалансированной

In [5]:
sum(y == 1) / len(y)

0.637

Все в порядке - положительный класс незначительно преобладает над отрицательным 

In [0]:
#nltk.download('wordnet')
#nltk.download('punkt')

In [0]:
#from nltk.stem import WordNetLemmatizer
#lemmatizer = WordNetLemmatizer()

Воспользуемся библиотекой string, чтобы преобразовать текст - удалить лишние пробелы и знаки пунктуации

In [0]:
def delete_punctuation(text):
  translation_table = str.maketrans(' ', ' ', string.punctuation)
  return text.translate(translation_table).lower()

In [0]:
def format_str(text):
  #word_list = nltk.word_tokenize(text)
  #words = [lemmatizer.lemmatize(word) for word in word_list]
  words = [word.strip() for word in delete_punctuation(text).split()]
  return ' '.join(words)

In [10]:
for i, line in tqdm(enumerate(X)):
  X[i] = format_str(line)

Для преобразования текстовых данных я использовал мешок слов (CountVectorizer) и TfidfTransformer. Для построения предсказаний по полученным признакам была использована логистическая регрессия. 

Для подбора оптимальных параметров, был реализован поиск по сетке.

In [0]:
#cv = KFold(shuffle=True, random_state=42)

In [0]:
#vectorizer = CountVectorizer()
#transformer = TfidfTransformer()
#clf = LinearSVC(loss='hinge', random_state=42)

In [0]:
#pipeline = Pipeline(steps=[('vectorizer', vectorizer), 
#                           ('transformer', transformer), 
#                           ('clf', clf)])

In [0]:
#gs = GridSearchCV(pipeline, {'vectorizer__ngram_range': [(1, 2), (1, 5)], 
#                             'vectorizer__max_df': [1, .9, .8], 
#                             'vectorizer__min_df': [1, .1, .2],
#                             'clf__C': np.arange(1, 2, .1),
#                             'clf__max_iter': np.arange(100, 500, 100),
#                             'clf__tol': np.linspace(0.001, 0.0001, 10)}, scoring='accuracy', cv=cv, verbose=0)
#gs.fit(X, y)
#gs.best_params_

In [0]:
vectorizer = CountVectorizer(ngram_range=(1, 5), max_df=0.9)
transformer = TfidfTransformer()
clf = LinearSVC(loss='hinge', tol=0.001, C=1.9, random_state=42, max_iter=400)

In [16]:
pipeline = Pipeline(steps=[('vectorizer', vectorizer), 
                           ('transformer', transformer), 
                           ('clf', clf)])
pipeline.fit(X, y)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.9,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 5), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('transformer',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LinearSVC(C=1.9, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
         

Импортируем тестовые данные

In [17]:
test_data = pd.read_csv('products_sentiment_test.tsv', sep='\t')
test_data.head()

,Id,text
0,0,"so , why the small digital elph , rather than ..."
1,1,3/4 way through the first disk we played on it...
2,2,better for the zen micro is outlook compatibil...
3,3,6 . play gameboy color games on it with goboy .
4,4,"likewise , i 've heard norton 2004 professiona..."


Достанем из данных индексы текстов. Они нам понадобятся позже для формирования сабмита

In [0]:
ids = test_data.Id.values
test_data.drop(labels=['Id'], axis=1, inplace=True)

Предобработаем текста из тестовых данных с помощью той же функции, которой мы обрабатывали обучающую выборку

In [19]:
X_test = test_data.text.values
for i, line in tqdm(enumerate(X_test)):
  X_test[i] = format_str(line)

Сделаем предсказания для тестовых данных

In [0]:
y_pred = pipeline.predict(X_test)

Формируем новый DataFrame, который будет нашим ответом

In [21]:
output = pd.DataFrame(zip(ids, y_pred), columns=['Id', 'y'], index=None)
output.head()

,Id,y
0,0,1
1,1,0
2,2,1
3,3,1
4,4,0


И заливаем свой ответ на Kaggle прямо из ноутбука

In [22]:
output.to_csv('submission.csv', index=None)
!kaggle competitions submit -c simplesentiment -f 'submission.csv' -m "Message"

100% 2.83k/2.83k [00:02<00:00, 1.15kB/s]
403 - Your team has used its submission allowance (5 of 5). This resets at midnight UTC (1.9 hours from now).
